In [84]:
import pandas as pd
import numpy as np
from datetime import datetime

In [85]:
cad = pd.read_csv('GMU_2021_Summer_CAD.csv')
avl = pd.read_csv('GMU_2021_Summer_AVL.csv')
avl.columns = ['AVL_IncidentNumber',
               'AVL_UnitID',
               'AVL_Timestamp',
               'AVL_UnitStatus',
               'AVL_Speed']
avl.dropna(subset = ['AVL_IncidentNumber'],
           inplace = True)
avl.reset_index(inplace = True, drop = True)
avl.isna().sum()

AVL_IncidentNumber    0
AVL_UnitID            0
AVL_Timestamp         0
AVL_UnitStatus        0
AVL_Speed             0
dtype: int64

In [87]:
# convert object to timestamp proper
avl['AVL_Timestamp'] = avl['AVL_Timestamp'].str.replace('ED', '')
cad['CAD_Timestamp'] = cad['CAD_Timestamp'].str.replace('ED', '')
avl['AVL_Timestamp'] = avl['AVL_Timestamp'].str.replace('ES', '')
cad['CAD_Timestamp'] = cad['CAD_Timestamp'].str.replace('ES', '')
avl['AVL_Timestamp'] = pd.to_datetime(avl['AVL_Timestamp'], format = '%Y%m%d%H%M%S')
cad['CAD_Timestamp'] = pd.to_datetime(cad['CAD_Timestamp'], format = '%Y%m%d%H%M%S')

In [88]:
#Create separate dat sets of ER and DP to then be joined to to form "transport" entry
cad_er = cad[cad['CAD_UnitStatus'].isin(['ER'])]
cad_dp= cad[cad['CAD_UnitStatus'].isin(['DP'])]
avl_er = avl[avl['AVL_UnitStatus'].isin(['ER'])]
avl_dp= avl[avl['AVL_UnitStatus'].isin(['DP'])]

In [89]:
#rename the columns so that that are labeled CAD or AVL and er or dp data frame
cad_er=cad_er.rename(columns={"CAD_IncidentNumber":"cad_er_IncidentNumber","CAD_UnitID":"cad_er_UnitID","CAD_Timestamp":"cad_er_Timestamp", "CAD_UnitStatus":"cad_er_UnitStatus"})
cad_dp=cad_dp.rename(columns={"CAD_IncidentNumber":"cad_dp_IncidentNumber","CAD_UnitID":"cad_dp_UnitID","CAD_Timestamp":"cad_dp_Timestamp", "CAD_UnitStatus":"cad_dp_UnitStatus"})

avl_er=avl_er.rename(columns={"AVL_IncidentNumber":"avl_er_IncidentNumber","AVL_UnitID":"avl_er_UnitID","AVL_Timestamp":"avl_er_Timestamp", "AVL_UnitStatus":"avl_er_UnitStatus", "AVL_Speed":"avl_er_Speed"})
avl_dp=avl_dp.rename(columns={"AVL_IncidentNumber":"avl_dp_IncidentNumber","AVL_UnitID":"avl_dp_UnitID","AVL_Timestamp":"avl_dp_Timestamp", "AVL_UnitStatus":"avl_dp_UnitStatus", "AVL_Speed":"avl_dp_Speed"})

In [92]:
#Create the FullKey to make sure the first time entry for the ER and DP status are being taken
cad_er['cad_er_FullKey'] = cad_er['cad_er_IncidentNumber'].map(str) +  cad_er['cad_er_UnitID'].map(str)
cad_dp['cad_dp_FullKey'] = cad_dp['cad_dp_IncidentNumber'].map(str) +  cad_dp['cad_dp_UnitID'].map(str)

avl_er['avl_er_FullKey'] = avl_er['avl_er_IncidentNumber'].map(str) +  avl_er['avl_er_UnitID'].map(str)
avl_dp['avl_dp_FullKey'] = avl_dp['avl_dp_IncidentNumber'].map(str) +  avl_dp['avl_dp_UnitID'].map(str)

#for validation note that there are currently duplicates in the data frames
dups_cad_er = cad_er.pivot_table(index = ['cad_er_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_cad_er)
dups_cad_dp = cad_dp.pivot_table(index = ['cad_dp_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_cad_dp)

dups_avl_er = avl_er.pivot_table(index = ['avl_er_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_avl_er)
dups_avl_dp = avl_dp.pivot_table(index = ['avl_dp_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_avl_dp)

cad_er_FullKey
E191120999M417     470
E180652335M424      92
E192510953E404      91
E200211385M418      90
E170101343M440      89
                  ... 
E181430071M437       1
E181430071R419       1
E181430081TT410      1
E181430091E421       1
E203662422M413       1
Length: 822765, dtype: int64
cad_dp_FullKey
E181160891E422B    9
E180441607T441     7
E180431602T440     6
E181591821A410E    6
E191582251A414E    6
                  ..
E181353295TT410    1
E181353327E429     1
E181353327T429     1
E181353332E427     1
E203662422M413     1
Length: 829830, dtype: int64
avl_er_FullKey
E191120999M417     477
E183130717M401E    455
E170333099E423     353
E182842013R439     350
E170333099E401     334
                  ... 
E181241800T405       1
E190731525M413       1
E190731454M413       1
E190731322M401       1
E203662422M413       1
Length: 711508, dtype: int64
avl_dp_FullKey
E180431163M435     69
E203610099M408B    34
E200390725E420     28
E171920096E422     28
E201111766E439     27
      

In [94]:
#sort both dataframes by timestamp so when dups are removed taking the first time stamp
cad_er.sort_values(by='cad_er_Timestamp', ascending=True)
cad_dp.sort_values(by='cad_dp_Timestamp', ascending=True)

avl_er.sort_values(by='avl_er_Timestamp', ascending=True)
avl_dp.sort_values(by='avl_dp_Timestamp', ascending=True)

#Remove duplicates from each dataframe, so each unit ID is only DP or ER to an incident once
cad_er_NEW = cad_er.drop_duplicates(subset='cad_er_FullKey', keep="first", inplace=False)
cad_dp_NEW = cad_dp.drop_duplicates(subset='cad_dp_FullKey', keep="first", inplace=False)

avl_er_NEW = avl_er.drop_duplicates(subset='avl_er_FullKey', keep="first", inplace=False)
avl_dp_NEW = avl_dp.drop_duplicates(subset='avl_dp_FullKey', keep="first", inplace=False)

In [95]:
#For Validation confirm that there are NOT currently duplicate values
dups_cad_er_NEW = cad_er_NEW.pivot_table(index = ['cad_er_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_cad_er_NEW)
dups_cad_dp_NEW = cad_dp_NEW.pivot_table(index = ['cad_dp_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_cad_dp_NEW)

dups_avl_er_NEW = avl_er_NEW.pivot_table(index = ['avl_er_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_avl_er_NEW)
dups_avl_dp_NEW = avl_dp_NEW.pivot_table(index = ['avl_dp_FullKey'], aggfunc ='size').sort_values(ascending=False)
print(dups_avl_dp_NEW)

cad_er_FullKey
70135E413          1
E192412172TL405    1
E192412119M438     1
E192412125E401     1
E192412125M401     1
                  ..
E181350940M439     1
E181350940R439     1
E181350947E415     1
E181350952E405     1
E203662422M413     1
Length: 822765, dtype: int64
cad_dp_FullKey
E170010002E410    1
E192402558E435    1
E192402493M418    1
E192402493M430    1
E192402493R421    1
                 ..
E181342009E422    1
E181342009M405    1
E181342010E415    1
E181342010M415    1
E203662422M413    1
Length: 829830, dtype: int64
avl_er_FullKey
E163662597E421    1
E192421902E417    1
E192421879E421    1
E192421879E430    1
E192421879M417    1
                 ..
E181210350M434    1
E181210369E414    1
E181210369M414    1
E181210369R419    1
E203662422M413    1
Length: 711508, dtype: int64
avl_dp_FullKey
E170010030E428     1
E200611859TT430    1
E200612028E419     1
E200612028M419     1
E200612129E410     1
                  ..
E182822249E418     1
E182822272E427     1
E182822329M411

In [98]:
#join the CAD data framaes together and then the AVL dataframes together
df_cad = cad_dp_NEW.merge(cad_er_NEW, 
               how = 'inner', 
               left_on = ['cad_dp_IncidentNumber',
                          'cad_dp_UnitID'],
               right_on = ['cad_er_IncidentNumber',
                           'cad_er_UnitID'],
               suffixes = ('cad_dp', 'cad_er'))

df_avl = avl_dp_NEW.merge(avl_er_NEW, 
               how = 'inner', 
               left_on = ['avl_dp_IncidentNumber',
                          'avl_dp_UnitID'],
               right_on = ['avl_er_IncidentNumber',
                           'avl_er_UnitID'],
               suffixes = ('avl_dp', 'avl_er'))

In [100]:
# create turnout col in merged dataframe
df_cad['cad_Timestamp_Turnout'] = abs(df_cad['cad_er_Timestamp'] - df_cad['cad_dp_Timestamp'])

df_avl['avl_Timestamp_Turnout'] = abs(df_avl['avl_er_Timestamp'] - df_avl['avl_dp_Timestamp'])

In [103]:
#convert the turnout time into seconds for simplier analysis
df_CADless60 = df_cad[df_cad['cad_Timestamp_Turnout'].dt.total_seconds() <= 60]
df_AVLless60 = df_avl[df_avl['avl_Timestamp_Turnout'].dt.total_seconds() <= 60]

In [110]:
#Different Analytics
df_cad.info()
df_CADless60.info()
516138/822376
#about 63% of incidents had a turnout time of 60 seconds or less in CAD dataset

df_avl.info()
df_AVLless60.info()
23061/44420
#only about 52% of incidents had a turnout time of 60 seconds or less in CAD dataset

<class 'pandas.core.frame.DataFrame'>
Int64Index: 822376 entries, 0 to 822375
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype          
---  ------                 --------------   -----          
 0   cad_dp_IncidentNumber  822376 non-null  object         
 1   cad_dp_UnitID          822376 non-null  object         
 2   cad_dp_Timestamp       822376 non-null  datetime64[ns] 
 3   cad_dp_UnitStatus      822376 non-null  object         
 4   cad_dp_FullKey         822376 non-null  object         
 5   cad_er_IncidentNumber  822376 non-null  object         
 6   cad_er_UnitID          822376 non-null  object         
 7   cad_er_Timestamp       822376 non-null  datetime64[ns] 
 8   cad_er_UnitStatus      822376 non-null  object         
 9   cad_er_FullKey         822376 non-null  object         
 10  cad_Timestamp_Turnout  822376 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), object(8), timedelta64[ns](1)
memory usage: 75.3+ MB
<class 'pandas.c

0.5191580369203062

In [121]:
df_avl["avl_Timestamp_Turnout"].mean()

Timedelta('0 days 00:03:37.305920756')

In [122]:
df_cad["cad_Timestamp_Turnout"].mean()

Timedelta('0 days 00:01:17.788538332')

In [123]:
df_cad["cad_Timestamp_Turnout"].max()

Timedelta('11 days 05:19:44')

In [124]:
df_avl["avl_Timestamp_Turnout"].max()

Timedelta('11 days 05:24:47')

In [125]:
df_cad["cad_Timestamp_Turnout"].min()

Timedelta('0 days 00:00:00')

In [126]:
df_avl["avl_Timestamp_Turnout"].min()

Timedelta('0 days 00:00:00')

In [128]:
df_cad[df_cad['cad_Timestamp_Turnout'].isin(['0 days 00:00:00'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28855 entries, 45 to 822343
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   cad_dp_IncidentNumber  28855 non-null  object         
 1   cad_dp_UnitID          28855 non-null  object         
 2   cad_dp_Timestamp       28855 non-null  datetime64[ns] 
 3   cad_dp_UnitStatus      28855 non-null  object         
 4   cad_dp_FullKey         28855 non-null  object         
 5   cad_er_IncidentNumber  28855 non-null  object         
 6   cad_er_UnitID          28855 non-null  object         
 7   cad_er_Timestamp       28855 non-null  datetime64[ns] 
 8   cad_er_UnitStatus      28855 non-null  object         
 9   cad_er_FullKey         28855 non-null  object         
 10  cad_Timestamp_Turnout  28855 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), object(8), timedelta64[ns](1)
memory usage: 2.6+ MB


In [129]:
df_avl[df_avl['avl_Timestamp_Turnout'].isin(['0 days 00:00:00'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 998 to 36197
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  40 non-null     object         
 1   avl_dp_UnitID          40 non-null     object         
 2   avl_dp_Timestamp       40 non-null     datetime64[ns] 
 3   avl_dp_UnitStatus      40 non-null     object         
 4   avl_dp_Speed           40 non-null     object         
 5   avl_dp_FullKey         40 non-null     object         
 6   avl_er_IncidentNumber  40 non-null     object         
 7   avl_er_UnitID          40 non-null     object         
 8   avl_er_Timestamp       40 non-null     datetime64[ns] 
 9   avl_er_UnitStatus      40 non-null     object         
 10  avl_er_Speed           40 non-null     object         
 11  avl_er_FullKey         40 non-null     object         
 12  avl_Timestamp_Turnout  40 non-null     timedelt

In [130]:
df_cad[df_cad['cad_Timestamp_Turnout'].isin(['11 days 05:19:44'])]

,cad_dp_IncidentNumber,cad_dp_UnitID,cad_dp_Timestamp,cad_dp_UnitStatus,cad_dp_FullKey,cad_er_IncidentNumber,cad_er_UnitID,cad_er_Timestamp,cad_er_UnitStatus,cad_er_FullKey,cad_Timestamp_Turnout
206080,E170333099,R419,2017-02-14 02:30:56,DP,E170333099R419,E170333099,R419,2017-02-02 21:11:12,ER,E170333099R419,11 days 05:19:44


In [131]:
df_avl[df_avl['avl_Timestamp_Turnout'].isin(['11 days 05:24:47'])]

,avl_dp_IncidentNumber,avl_dp_UnitID,avl_dp_Timestamp,avl_dp_UnitStatus,avl_dp_Speed,avl_dp_FullKey,avl_er_IncidentNumber,avl_er_UnitID,avl_er_Timestamp,avl_er_UnitStatus,avl_er_Speed,avl_er_FullKey,avl_Timestamp_Turnout
14726,E170333099,R419,2017-02-14 02:37:33,DP,not moving,E170333099R419,E170333099,R419,2017-02-02 21:12:46,ER,moving,E170333099R419,11 days 05:24:47


In [132]:
cad_SORT = cad.sort_values('CAD_Timestamp').groupby(['CAD_IncidentNumber','CAD_UnitID','CAD_UnitStatus'])['CAD_Timestamp'].agg(['first'])

In [133]:
df_cad

,cad_dp_IncidentNumber,cad_dp_UnitID,cad_dp_Timestamp,cad_dp_UnitStatus,cad_dp_FullKey,cad_er_IncidentNumber,cad_er_UnitID,cad_er_Timestamp,cad_er_UnitStatus,cad_er_FullKey,cad_Timestamp_Turnout
0,E200890111,E430,2020-03-29 01:35:54,DP,E200890111E430,E200890111,E430,2020-03-29 01:37:33,ER,E200890111E430,0 days 00:01:39
1,E182263037,M414,2018-08-14 22:39:08,DP,E182263037M414,E182263037,M414,2018-08-14 22:40:21,ER,E182263037M414,0 days 00:01:13
2,E191072366,E405,2019-04-17 19:23:23,DP,E191072366E405,E191072366,E405,2019-04-17 19:24:22,ER,E191072366E405,0 days 00:00:59
3,E172781213,E405,2017-10-05 10:27:15,DP,E172781213E405,E172781213,E405,2017-10-05 10:28:13,ER,E172781213E405,0 days 00:00:58
4,E202371418,E421,2020-08-24 14:28:10,DP,E202371418E421,E202371418,E421,2020-08-24 14:28:37,ER,E202371418E421,0 days 00:00:27
...,...,...,...,...,...,...,...,...,...,...,...
822371,E192481812,M411B,2019-09-05 13:34:08,DP,E192481812M411B,E192481812,M411B,2019-09-05 13:35:05,ER,E192481812M411B,0 days 00:00:57
822372,E201811823,M402,2020-06-29 17:25:12,DP,E201811823M402,E201811823,M402,2020-06-29 17:26:00,ER,E201811823M402,0 days 00:00:48
822373,E173070886,E402,2017-11-03 08:24:23,DP,E173070886E402,E173070886,E402,2017-11-03 08:25:36,ER,E173070886E402,0 days 00:01:13
822374,E193442345,M438,2019-12-10 17:11:34,DP,E193442345M438,E193442345,M438,2019-12-10 17:11:38,ER,E193442345M438,0 days 00:00:04


In [138]:
df_ALL = df_avl.merge(df_cad, 
               how = 'inner', 
               left_on = ['avl_er_FullKey'],
               right_on = ['cad_er_FullKey'],
               suffixes = ('avl_er', 'cad_er'))

In [142]:
df_ALL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44415 entries, 0 to 44414
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  44415 non-null  object         
 1   avl_dp_UnitID          44415 non-null  object         
 2   avl_dp_Timestamp       44415 non-null  datetime64[ns] 
 3   avl_dp_UnitStatus      44415 non-null  object         
 4   avl_dp_Speed           44415 non-null  object         
 5   avl_dp_FullKey         44415 non-null  object         
 6   avl_er_IncidentNumber  44415 non-null  object         
 7   avl_er_UnitID          44415 non-null  object         
 8   avl_er_Timestamp       44415 non-null  datetime64[ns] 
 9   avl_er_UnitStatus      44415 non-null  object         
 10  avl_er_Speed           44415 non-null  object         
 11  avl_er_FullKey         44415 non-null  object         
 12  avl_Timestamp_Turnout  44415 non-null  timedel

In [141]:
df_ALL['CAD_vs_AVL_Turnout'] = abs(df_ALL['avl_Timestamp_Turnout'] - df_ALL['cad_Timestamp_Turnout'])

In [143]:
df_ALL['CAD_vs_AVL_Turnout'].mean()

Timedelta('0 days 00:02:45.606326691')

In [147]:
df_ALL['CAD_vs_AVL_Turnout'].min()

Timedelta('0 days 00:00:00')

In [150]:
equal = df_ALL[df_ALL['CAD_vs_AVL_Turnout'].isin(['0 days 00:00:00'])]

In [154]:
464/(44415)

0.010446921085218958

In [155]:
dfALL_CADless60 = df_ALL[df_ALL['cad_Timestamp_Turnout'].dt.total_seconds() <= 60]

In [156]:
dfALL_CADless60.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23997 entries, 2 to 44412
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  23997 non-null  object         
 1   avl_dp_UnitID          23997 non-null  object         
 2   avl_dp_Timestamp       23997 non-null  datetime64[ns] 
 3   avl_dp_UnitStatus      23997 non-null  object         
 4   avl_dp_Speed           23997 non-null  object         
 5   avl_dp_FullKey         23997 non-null  object         
 6   avl_er_IncidentNumber  23997 non-null  object         
 7   avl_er_UnitID          23997 non-null  object         
 8   avl_er_Timestamp       23997 non-null  datetime64[ns] 
 9   avl_er_UnitStatus      23997 non-null  object         
 10  avl_er_Speed           23997 non-null  object         
 11  avl_er_FullKey         23997 non-null  object         
 12  avl_Timestamp_Turnout  23997 non-null  timedel

In [157]:
dfALL_AVLless60 = df_ALL[df_ALL['avl_Timestamp_Turnout'].dt.total_seconds() <= 60]

In [158]:
dfALL_AVLless60.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23056 entries, 0 to 44413
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  23056 non-null  object         
 1   avl_dp_UnitID          23056 non-null  object         
 2   avl_dp_Timestamp       23056 non-null  datetime64[ns] 
 3   avl_dp_UnitStatus      23056 non-null  object         
 4   avl_dp_Speed           23056 non-null  object         
 5   avl_dp_FullKey         23056 non-null  object         
 6   avl_er_IncidentNumber  23056 non-null  object         
 7   avl_er_UnitID          23056 non-null  object         
 8   avl_er_Timestamp       23056 non-null  datetime64[ns] 
 9   avl_er_UnitStatus      23056 non-null  object         
 10  avl_er_Speed           23056 non-null  object         
 11  avl_er_FullKey         23056 non-null  object         
 12  avl_Timestamp_Turnout  23056 non-null  timedel

In [160]:
dfALL_less60 = df_ALL[df_ALL['CAD_vs_AVL_Turnout'].dt.total_seconds() <= 60]
dfALL_less60.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35106 entries, 1 to 44414
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  35106 non-null  object         
 1   avl_dp_UnitID          35106 non-null  object         
 2   avl_dp_Timestamp       35106 non-null  datetime64[ns] 
 3   avl_dp_UnitStatus      35106 non-null  object         
 4   avl_dp_Speed           35106 non-null  object         
 5   avl_dp_FullKey         35106 non-null  object         
 6   avl_er_IncidentNumber  35106 non-null  object         
 7   avl_er_UnitID          35106 non-null  object         
 8   avl_er_Timestamp       35106 non-null  datetime64[ns] 
 9   avl_er_UnitStatus      35106 non-null  object         
 10  avl_er_Speed           35106 non-null  object         
 11  avl_er_FullKey         35106 non-null  object         
 12  avl_Timestamp_Turnout  35106 non-null  timedel

In [161]:
dfALL_less30 = df_ALL[df_ALL['CAD_vs_AVL_Turnout'].dt.total_seconds() <= 30]
dfALL_less30.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24034 entries, 1 to 44414
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  24034 non-null  object         
 1   avl_dp_UnitID          24034 non-null  object         
 2   avl_dp_Timestamp       24034 non-null  datetime64[ns] 
 3   avl_dp_UnitStatus      24034 non-null  object         
 4   avl_dp_Speed           24034 non-null  object         
 5   avl_dp_FullKey         24034 non-null  object         
 6   avl_er_IncidentNumber  24034 non-null  object         
 7   avl_er_UnitID          24034 non-null  object         
 8   avl_er_Timestamp       24034 non-null  datetime64[ns] 
 9   avl_er_UnitStatus      24034 non-null  object         
 10  avl_er_Speed           24034 non-null  object         
 11  avl_er_FullKey         24034 non-null  object         
 12  avl_Timestamp_Turnout  24034 non-null  timedel

In [164]:
dfALL_less1 = df_ALL[df_ALL['CAD_vs_AVL_Turnout'].dt.total_seconds() <= 1]
dfALL_less1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1342 entries, 56 to 44397
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   avl_dp_IncidentNumber  1342 non-null   object         
 1   avl_dp_UnitID          1342 non-null   object         
 2   avl_dp_Timestamp       1342 non-null   datetime64[ns] 
 3   avl_dp_UnitStatus      1342 non-null   object         
 4   avl_dp_Speed           1342 non-null   object         
 5   avl_dp_FullKey         1342 non-null   object         
 6   avl_er_IncidentNumber  1342 non-null   object         
 7   avl_er_UnitID          1342 non-null   object         
 8   avl_er_Timestamp       1342 non-null   datetime64[ns] 
 9   avl_er_UnitStatus      1342 non-null   object         
 10  avl_er_Speed           1342 non-null   object         
 11  avl_er_FullKey         1342 non-null   object         
 12  avl_Timestamp_Turnout  1342 non-null   timedel